In [ ]:
# Practical Exam: House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


In [ ]:
# Importing the necessary libraries
import pandas as pd
import numpy as np



In [ ]:
# Loading the dataset
file_path_sales = 'Datasets/Real Estate/house_sales.csv'
file_path_train = 'Datasets/Real Estate/train.csv'
file_path_validation = 'Datasets/Real Estate/validation.csv'

house_sales = pd.read_csv(file_path_sales)

# getting dtypes of the columns
house_sales.dtypes

# Task 1

The team at RealAgents knows that the city that a property is located in makes a difference to the sale price. 

Unfortuntately they believe that this isn't always recorded in the data. 

Calculate the number of missing values of the `city`. 

 - You should use the data in the file "house_sales.csv". 

 - Your output should be an object `missing_city`, that contains the number of missing values in this column. 

In [ ]:
# Calculating the number of missing values in the 'city' column
# count the number of city values that are equal to '--'
missing_city = house_sales['city'].value_counts()['--']
missing_city

# Task 2 

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "house_sales.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [ ]:
# working on task 2
# creating a copy of the original dataframe
clean_data = house_sales.copy()

# replacing missing values in the 'city' column with 'Unknown'. The missing values are represented by '--'
clean_data['city'] = clean_data['city'].replace('--', 'Unknown')

# check for missing values in the 'sale_price' column
missing_sale_price = clean_data['sale_price'].isnull().sum()

# check for negative values in the 'sale_price' column
negative_sale_price = clean_data['sale_price'][clean_data['sale_price'] < 0].count()

# working on sales_date column. check for missing values
missing_sales_date = clean_data['sale_date'].isnull().sum()

# working on months_listed column. check for missing values. 
missing_months_listed = clean_data['months_listed'].isnull().sum()

# replace missing values in the 'months_listed' column with the mean number of months listed, to one decimal place
clean_data['months_listed'] = clean_data['months_listed'].fillna(clean_data['months_listed'].mean())

# make sure that the 'months_listed' column is rounded to one decimal place
clean_data['months_listed'] = clean_data['months_listed'].round(1)

# working on bedrooms column. check for missing values
missing_bedrooms = clean_data['bedrooms'].isnull().sum()

# working on house_type column. check for missing values.
missing_house_type = clean_data['house_type'].isnull().sum()

# fixing variables in house_type column. replace 'Det.' with 'Detached' and 'Semi' with 'Semi-detached', 'Terr.' with 'Terraced'
clean_data['house_type'] = clean_data['house_type'].replace('Det.', 'Detached')
clean_data['house_type'] = clean_data['house_type'].replace('Semi', 'Semi-detached')
clean_data['house_type'] = clean_data['house_type'].replace('Terr.', 'Terraced')

# getting unique values in the house_type column
house_type_unique = clean_data['house_type'].unique()

# dropping the ' sq.m.' from the 'area' column. Striping the column of the ' sq.m.' string and converting the column to float
clean_data['area'] = clean_data['area'].str.strip(' sq.m.').astype(float)

# check for missing values in the 'area' column
missing_area = clean_data['area'].isnull().sum()

# calc average of area
average_area = clean_data['area'].mean()

# viewing cleaned data
clean_data



# Task 3 

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price. 

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`. 

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`. 

 - Your answers should be rounded to 1 decimal place.   

In [ ]:
# viewing house_sales dataframe
# creating a pivto table to show the average sale_price by the number of bedrooms and the variance by the number of bedrooms. the table will be called price_by_rooms
price_by_rooms = house_sales.pivot_table(index='bedrooms', values='sale_price', aggfunc=[np.mean, np.var])

# renaming the columns
price_by_rooms.columns = ['avg_price', 'var_price']

# rounding the average sale price and variance to one decimal place
price_by_rooms = price_by_rooms.round(1)

# viewing the price_by_rooms dataframe
price_by_rooms

# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [ ]:
# working on task 4
# importing the necessary libraries
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

train = pd.read_csv(file_path_train)
test = pd.read_csv(file_path_validation)


# getting unique values in the 'house_type' column and city column
house_type_unique = train['house_type'].unique()
city_unique = train['city'].unique()

# get dummies for the 'house_type' and 'city' columns
train = pd.get_dummies(train, columns=['house_type', 'city'], drop_first=True)
test = pd.get_dummies(test, columns=['house_type', 'city'], drop_first=True)

# get the dtypes of the columns for train
train.dtypes, test.dtypes

# Convert the 'sales_date' column to datetime
train['sale_date'] = pd.to_datetime(train['sale_date'])
test['sale_date'] = pd.to_datetime(test['sale_date'])

# Find the minimum date in the training set
min_date = train['sale_date'].min()

# Create new columns for the number of days since the minimum date
train['days_since_min_date'] = (train['sale_date'] - min_date).dt.days
test['days_since_min_date'] = (test['sale_date'] - min_date).dt.days

# Now you can drop the original 'sales_date' column
train = train.drop(columns=['sale_date'])
test = test.drop(columns=['sale_date'])

train

In [ ]:

# defining the features and target variable. y is the target variable and its is the sale_price column. x is the features, it should be all the columns except the sale_price column
X_train = train.drop(['sale_price'], axis=1)
y_train = train['sale_price']

X_test = test

# fitting the model
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# making predictions
y_pred = lin_reg.predict(X_test)

# creating the base_result dataframe
base_result = pd.DataFrame({'house_id': test['house_id'], 'price': y_pred})

# viewing the base_result dataframe
base_result





# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [ ]:
# working on task 5: fitting a comparison model which random forest regressor
from sklearn.ensemble import RandomForestRegressor

# fitting the model
rf = RandomForestRegressor(random_state=9)
rf.fit(X_train, y_train)

# making predictions
y_pred_rf = rf.predict(X_test)

# creating the compare_result dataframe
compare_result = pd.DataFrame({'house_id': test['house_id'], 'price': y_pred_rf})

# viewing the compare_result dataframe
compare_result
